## Sentence-level online prompty mining: XORQA

In [1]:
import copy
import re
import os, sys
import json
import glob
from collections import Counter, defaultdict
import pandas as pd
import jsonlines

from exploring_sentence_level import (
    load_model,
    mine_prompt_gt,  
    segment_sentence,
    run_online_prompt_mining
)

### 0. Download dataset

```bash
cd ../scripts
bash ./download_xorqa.sh
```

### 1. Process dataset

In [2]:
XORQA_BASE_DIR = '../data/xorqa/en/tydi_xor_gp/'
xorqa_xx = {
    'train': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_train_data.json'), 'r'))['data'],
      'val': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_dev_data.json'), 'r'))['data'],   
}

In [3]:
xorqa_xx['train'][-1]

{'title': 'title:Vamsy_parentSection:Introduction_sectionName:Career._sectionIndex:2',
 'paragraphs': [{'context': 'He has published a short stories compilation called "Maa Pasalapudi Kathalu". Besides that compilation, Vamsy has written a wide variety of short stories since 1974 when he was 18 years old. His major works include "Mahallo kokila", "Manchupallaki", "Aa Naati Vaana Chinukulu", "Venditera Kathalu" (original scripts of "Sankarabharanam" and "Anveshana"), "Vennela Bomma", "Gokulam lo Radha", "Ravvala konda", "Sree seetarama lanchi service Rajahmundry", "Manyam rani", "Rangularatnam". He has penned around 150 short stories published in swathi weekly under title "Maa Diguwa Godavari Kathalu" For his contributions to the art of story telling with a native approach through his books he was bestowed with "Sripada Puraskhaaram" at Rajamundry on 17 April 2011.',
   'qas': [{'question': 'మా పసలపూడి కథలు పుస్తకమును ఎవరు రచించారు?',
     'answers': [{'text': 'Vamsy', 'answer_start': 1

In [4]:
def get_xorqa_answer_str(context, qas):
    context_qa_pairs = []
    for qa in qas:
        question = qa['question']
        lang = qa['lang']
        answer = qa['answers'][0]['text']
        answer_start = qa['answers'][0]['answer_start']
        context_qa_pairs.append((context, question, answer, answer_start, lang))
    return context_qa_pairs

In [5]:
xorqa_xx_dataset = defaultdict(lambda: { 'train': [], 'val': [] })

for split_name in ['train', 'val']:
    for i, item in enumerate(xorqa_xx[split_name]):
        paragraphs = item['paragraphs']
#         print('.' ,end='')
        for j, paragraph in enumerate(paragraphs):

            context = paragraph['context']
            context_qa_pairs = get_xorqa_answer_str(context=context, qas=paragraph['qas'])

            for context_qa_pair in context_qa_pairs:
                context, question, answer, answer_start, lang = context_qa_pair
                gt_sentence = mine_prompt_gt((context, question, answer, answer_start))
                qa_item = {
                     'question': question,
                     'lang': lang,
                     'context': context,
                     'segmented_context': segment_sentence(context),
                     'answer': answer,
                     'answer_start': answer_start,
                     'gt_sentence': gt_sentence,
                }
                xorqa_xx_dataset[lang][split_name].append(qa_item)

In [6]:
list(xorqa_xx_dataset.keys())

['bn', 'ja', 'ko', 'ru', 'fi', ' ar', 'te', 'ar']

In [7]:
len(xorqa_xx_dataset['ar']['val'])

485

### 2. Compute question-sentence similarity


#### 2.1 Load models

##### a) Load mUSE_small (v3) model (as a baseline)

In [8]:
muse_small_v3_model = load_model('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')


##### b) Load teacher models

In [9]:
XQUAD_TEACHER_DIR = '../../../../CL-ReLKT_store/models/XQUAD/teacher_model/'
MLQA_TEACHER_DIR = '../../../../CL-ReLKT_store/models/MLQA/teacher_model/'

In [10]:
xquad_teacher_model = load_model(XQUAD_TEACHER_DIR)
mlqa_teacher_model = load_model(MLQA_TEACHER_DIR)

##### c) Load student models

In [11]:
XQUAD_STUDENT_SUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/XQUAD/student_best_supported_languages/'
XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/XQUAD/student_best_unsupported_languages/'

XORQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/XORQA/student_best_supported_languages/'
XORQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/XORQA/student_best_unsupported_languages/'

MLQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/MLQA/student_best_supported_languages/'
MLQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../../CL-ReLKT_store/models/MLQA/student_best_unsupported_languages/'

In [12]:
xquad_student_supported_langs_model = load_model(XQUAD_STUDENT_SUPPORTED_LANGS_DIR)
xorqa_student_supported_langs_model = load_model(XORQA_STUDENT_SUPPORTED_LANGS_DIR)
mlqa_student_supported_langs_model = load_model(MLQA_STUDENT_SUPPORTED_LANGS_DIR)

xquad_student_unsupported_langs_model = load_model(XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR)
xorqa_student_unsupported_langs_model = load_model(XORQA_STUDENT_UNSUPPORTED_LANGS_DIR)
mlqa_student_unsupported_langs_model = load_model(MLQA_STUDENT_UNSUPPORTED_LANGS_DIR)

In [13]:
MODEL_MAPPING = {
  # mUSE_small
  'model-muse_small_v3': muse_small_v3_model,
  # teacher    
  'model-xquad_teacher': xquad_teacher_model,
  'model-mlqa_teacher': mlqa_teacher_model,
  # student
  'model-xquad_student_supported_langs': xquad_student_supported_langs_model,
  'model-xorqa_student_supported_langs': xorqa_student_supported_langs_model,
  'model-mlqa_student_supported_langs': mlqa_student_supported_langs_model,
  'model-xquad_student_unsupported_langs': xquad_student_unsupported_langs_model,
  'model-xorqa_student_unsupported_langs': xorqa_student_unsupported_langs_model,
  'model-mlqa_student_unsupported_langs': mlqa_student_unsupported_langs_model,
}



In [14]:
DATASET_MAPPING = {}

for lang in list(xorqa_xx_dataset.keys()):
    if len(xorqa_xx_dataset[lang]['train']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_train'] = xorqa_xx_dataset[lang]['train']
    if len(xorqa_xx_dataset[lang]['val']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_val'] = xorqa_xx_dataset[lang]['val']
print(DATASET_MAPPING.keys())

dict_keys(['dataset-xorqa_bn_train', 'dataset-xorqa_bn_val', 'dataset-xorqa_ja_train', 'dataset-xorqa_ja_val', 'dataset-xorqa_ko_train', 'dataset-xorqa_ko_val', 'dataset-xorqa_ru_train', 'dataset-xorqa_ru_val', 'dataset-xorqa_fi_train', 'dataset-xorqa_fi_val', 'dataset-xorqa_ar_train', 'dataset-xorqa_te_train', 'dataset-xorqa_te_val', 'dataset-xorqa_ar_val'])


#### 2.2 Run inference and evaluate

The following function `run_online_prompt_mining` iterates over question-answer-passage triplets $(q_i, a_i, p_i)$ and compute 
the cosine similarity scores between question $q_i$ and segmented setences $s^i_j \textrm{ where } p_i = ( s^i_0, \ldots , s^i_{|p_i| - 1} )$ , and rank each quesiton-sentence pair by similairy score. Then, it evaluate the sentence-level precision@k.  Note: There is only 1 groundtruth sentence (i.e. the sentence where the answer span is a part of). 


In [ ]:
results = defaultdict(lambda : defaultdict())

for dataset_prefix, dataset in DATASET_MAPPING.items():
    print(f'\n\ndataset_prefix: {dataset_prefix}')
    for model_prefix, model in MODEL_MAPPING.items():
        
        print(f'\n - model_prefix: {model_prefix}')
        prefix = f'{dataset_prefix}+{model_prefix}'
        _result = run_online_prompt_mining(dataset,
                             prefix=f'{dataset_prefix}_{model_prefix}',
                             model=model)


        results[dataset_prefix][model_prefix] = _result
        print('--'*50)
    print('\n')    
    print('=='*50)
    print('\n')    
    



dataset_prefix: dataset-xorqa_bn_train

 - model_prefix: model-muse_small_v3


100%|██████████| 2474/2474 [05:04<00:00,  8.11it/s]



	Evaluation result:
	 - Accuracy: 0.4062
	 - precision_at_k:
{1: 0.40622473726758285,
 2: 0.6471301535974131,
 3: 0.7934518997574778,
 4: 0.8811641067097817,
 5: 0.9329021827000809,
 6: 0.9595796281325788,
 7: 0.97696038803557,
 8: 0.9854486661277284,
 9: 0.9894907033144705,
 10: 0.9911075181891673}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-xquad_teacher


100%|██████████| 2474/2474 [05:00<00:00,  8.24it/s]



	Evaluation result:
	 - Accuracy: 0.3868
	 - precision_at_k:
{1: 0.3868229587712207,
 2: 0.6329830234438156,
 3: 0.778496362166532,
 4: 0.8779304769603881,
 5: 0.9240097008892482,
 6: 0.9583670169765561,
 7: 0.9737267582861763,
 8: 0.9834276475343573,
 9: 0.9878738884397736,
 10: 0.9907033144704931}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-mlqa_teacher


...

100%|██████████| 2474/2474 [04:58<00:00,  8.28it/s]



	Evaluation result:
	 - Accuracy: 0.3395
	 - precision_at_k:
{1: 0.3395311236863379,
 2: 0.5905416329830234,
 3: 0.7623282134195635,
 4: 0.862570735650768,
 5: 0.9211802748585287,
 6: 0.952303961196443,
 7: 0.9717057396928052,
 8: 0.9814066289409863,
 9: 0.9894907033144705,
 10: 0.9927243330638642}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-xorqa_student_unsupported_langs


100%|██████████| 2474/2474 [04:39<00:00,  8.84it/s]



	Evaluation result:
	 - Accuracy: 0.2676
	 - precision_at_k:
{1: 0.2675828617623282,
 2: 0.5161681487469685,
 3: 0.6956345998383185,
 4: 0.8096200485044462,
 5: 0.8848019401778496,
 6: 0.9268391269199676,
 7: 0.9567502021018593,
 8: 0.9725141471301536,
 9: 0.9785772029102667,
 10: 0.9830234438156831}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-mlqa_student_unsupported_langs


 61%|██████▏   | 1521/2474 [03:01<02:01,  7.86it/s]

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


#### 2.3 Write result as JSON file

In [ ]:
json.dump(results, open('./eval_results.dataset_name-xorqa.json', 'w'), ensure_ascii=False, indent=2)

### 3. Convert evaluation results to a pandas.DataFrame

In [18]:
results = json.load(open('./eval_results.dataset_name-xorqa.json', 'r'))

In [19]:
list(results.keys()), len(list(results.keys()))

(['dataset-xorqa_bn_train',
  'dataset-xorqa_bn_val',
  'dataset-xorqa_ja_train',
  'dataset-xorqa_ja_val',
  'dataset-xorqa_ko_train',
  'dataset-xorqa_ko_val',
  'dataset-xorqa_ru_train',
  'dataset-xorqa_ru_val',
  'dataset-xorqa_fi_train',
  'dataset-xorqa_fi_val',
  'dataset-xorqa_ar_train',
  'dataset-xorqa_te_train',
  'dataset-xorqa_te_val',
  'dataset-xorqa_ar_val'],
 14)

In [22]:
result_objs = []
for dataset_name, result_model_group in results.items():
    for model_name, (metric, raw_result) in result_model_group.items():
        top1, precision_at_k = metric
        
        result_objs.append({
            'dataset_name': dataset_name,
            'model_name': model_name,
            'precision_at_1': top1,
            'precision_at_2': precision_at_k['2'],
            'precision_at_3': precision_at_k['6'],
            'precision_at_4': precision_at_k['4'],
            'precision_at_5': precision_at_k['5'],
            'precision_at_10': precision_at_k['10'],
        })
    
df = pd.DataFrame.from_dict(result_objs)
df.to_csv('./eval_results.dataset_name-xorqa.csv')

In [23]:
df

,dataset_name,model_name,precision_at_1,precision_at_2,precision_at_3,precision_at_4,precision_at_5,precision_at_10
0,dataset-xorqa_bn_train,model-muse_small_v3,0.406225,0.647130,0.959580,0.881164,0.932902,0.991108
1,dataset-xorqa_bn_train,model-xquad_teacher,0.386823,0.632983,0.958367,0.877930,0.924010,0.990703
2,dataset-xorqa_bn_train,model-mlqa_teacher,0.422797,0.650768,0.959580,0.881568,0.928456,0.992320
3,dataset-xorqa_bn_train,model-xquad_student_supported_langs,0.310833,0.567502,0.947049,0.853678,0.905821,0.988682
4,dataset-xorqa_bn_train,model-xorqa_student_supported_langs,0.301536,0.564268,0.940582,0.848019,0.904204,0.986661
...,...,...,...,...,...,...,...,...
121,dataset-xorqa_ar_val,model-xorqa_student_supported_langs,0.494845,0.721649,0.975258,0.917526,0.958763,0.995876
122,dataset-xorqa_ar_val,model-mlqa_student_supported_langs,0.540206,0.748454,0.975258,0.931959,0.967010,0.989691
123,dataset-xorqa_ar_val,model-xquad_student_unsupported_langs,0.404124,0.628866,0.956701,0.880412,0.925773,1.000000
124,dataset-xorqa_ar_val,model-xorqa_student_unsupported_langs,0.453608,0.717526,0.975258,0.921649,0.962887,0.985567
